# Mathematical Formulation and Implementation of Dynamic Task Assignment Problem in 10 Minute School
Code and definition of a dynamic task assignment problem (Mathematical Formulation and Implementation of Dynamic Task Assignment Problem in 10 Minute School) for Operations Management Lab.

# **Problem Description:**
An experienced operations manager is responsible for assigning tasks to a team of six workers (workers 1 to 6) to maximize the output level. Each day, he receives a list of tasks with various characteristics, including task type, count, priority, urgency, time requirement, and quality scores of workers for each task type. The goal is to efficiently assign these tasks to workers while considering their availability, task counts, and the importance of the tasks.

Details can be found here: [**GitHub**](https://github.com/azminewasi/DynamicTaskAssignmentProblemOM)

# **Mathematical Formulation:**

## **Indices:**
- $i$: Index for workers, $i \in \{1, 2, 3, 4, 5, 6\}$.
- $j$: Index for tasks, $j \in \{1, 2, \ldots, N\}$, where $N$ is the total number of tasks.

## **Parameters:**
- $T_j$: Task type of task $j$.
- $C_j$: Count of task $j$.
- $P_j$: Priority of task $j$.
- $U_j$: Urgency of task $j$.
- $Q_{ij}$: Quality score of worker $i$ for task type $T_j$.
- $R_j$: Time requirement for task $j$.
- $H_i$: Available working hours per worker $i$.

## **Variables:**
- $X_{ij}$: Binary decision variable indicating whether task $j$ is assigned to worker $i$.

## **Objective Function:**
The objective is to maximize the total output level, which is calculated as the sum of quality scores for each assigned task, weighted by its priority and urgency. The higher the quality score, priority, and urgency, the more critical the task, and it contributes more to the total output level.


$\text{Maximize: } \sum_{i=1}^{6} \sum_{j=1}^{N} X_{ij} \cdot Q_{ij} \cdot P_j \cdot U_j$


This objective function reflects the primary goal of maximizing the overall performance by efficiently assigning tasks to workers based on their quality scores and task characteristics.

## **Constraints:**

1. **Task Assignment Constraint:**
Each task $j$ can be assigned at most once. This constraint ensures that a task is either assigned to a worker or left unassigned.

$\sum_{i=1}^{6} X_{ij} \leq 1 \quad \forall j \in \{1, 2, \ldots, N\}$

2. **Worker Availability Constraint:**
Each worker $i$ can work at most $H_i$ hours per day. This constraint ensures that workers do not exceed their available working hours.

$\sum_{j=1}^{N} X_{ij} \cdot R_j \leq H_i \quad \forall i \in \{1, 2, 3, 4, 5, 6\}$

3. Task Count Constraint:**
Ensure that the total assigned count of each task $j$ matches its given count. This constraint ensures that the assigned task count matches the provided task count for each task type.

$\sum_{i=1}^{6} X_{ij} \cdot C_j = C_j \quad \forall j \in \{1, 2, \ldots, N\}$

**Binary Variable:**

The decision variable $X_{ij}$ is binary and takes a value of 1 if task $j$ is assigned to worker $i$ and 0 if it is not assigned.


# **Explanation:**
This mathematical formulation addresses the operational challenge of assigning tasks to workers to maximize productivity. The objective function seeks to maximize the output level, which is a combination of the quality scores, task priorities, and urgencies. By optimizing these factors, you can ensure that the most important and time-critical tasks are assigned to the most skilled workers.
The constraints ensure that tasks are not over-allocated, workers do not exceed their working hours, and the assigned task counts match the given counts. This formulation strikes a balance between productivity and resource constraints.
Solving this optimization problem will lead to task assignments that achieve the highest possible output level, ultimately improving the operational efficiency of your team.

# 1. Import Modules

In [ ]:
import pandas as pd
from io import StringIO

# 2. Define Inputs

In [ ]:
# Define task time requirements
task_time_requirements = {
    1: 4,
    2: 6,
    3: 1,
    4: 5,
    5: 2,
    6: 3
}

In [ ]:
# Define the quality scores for each worker and task type
data = """
task,worker1,worker2,worker3,worker4,worker5,worker6
task1,5,3,1,2,3,4
task2,5,3,2,2,1,2
task3,1,3,5,5,3,4
task4,3,3,2,2,3,3
task5,1,3,1,4,4,4
task6,1,1,5,5,2,1
"""

# Create a DataFrame
quality_df = pd.read_csv(StringIO(data))
quality_df = quality_df.drop(columns='task')

quality_df

In [ ]:
# Define available working hours per worker
available_hours_per_worker = 8

In [ ]:
# Define the list of tasks with their counts, priorities, and urgencies
# With columns - task_no,task_type,count,priority,urgency

data = """
task_no,task_type,count,priority,urgency
1,3,9,2,5
2,1,2,1,3
3,5,4,2,4
4,5,3,4,2
5,1,5,5,3
6,4,6,3,1
7,4,4,3,5
"""

# Create a DataFrame
tasks_df  = pd.read_csv(StringIO(data))
tasks_df

# 3. Assignment Algoprithm

In [ ]:
# Initialize a dictionary to track assigned tasks and free time per worker
assigned_tasks = {worker: [] for worker in quality_df.columns}
free_time_per_worker = {worker: available_hours_per_worker for worker in quality_df.columns}

# Sort tasks by priority (descending), urgency (descending), and time requirement (ascending)
tasks_df = tasks_df.sort_values(by=["priority", "urgency", "task_type"], ascending=[False, False, True])

# Create a dictionary to keep track of task counts for each unique combination of task type, priority, and urgency
given_task_counts = {}


In [ ]:

# Loop through tasks and assign them to workers to maximize output level
for _, row in tasks_df.iterrows():
    task_type = row["task_type"]
    task_no = row["task_no"]
    priority = row["priority"]
    urgency = row["urgency"]

    # Check if the task type, priority, and urgency combination is in the given_task_counts dictionary
    key = (task_type, priority, urgency)
    if key in given_task_counts:
        count = given_task_counts[key]
    else:
        count = row["count"]
        given_task_counts[key] = count

    time_required = task_time_requirements[task_type]

    # Sort workers by quality score for the given task type
    sorted_workers = quality_df.columns.to_list()
    sorted_workers.sort(key=lambda worker: -quality_df.at[task_type, worker])

    # Try to assign the task to available workers
    for worker in sorted_workers:
        for worker in sorted_workers:
            if count > 0 and free_time_per_worker[worker] >= time_required:
                assigned_tasks[worker].append((task_no, task_type, time_required, priority, urgency))
                free_time_per_worker[worker] -= time_required
                count -= 1


In [ ]:

# Calculate the total output level
total_output_level = 0
for worker, tasks in assigned_tasks.items():
    for task_no, task_type, time_required, priority, urgency in tasks:
        quality_score = quality_df.at[task_type, worker]
        output_level = quality_score * 1 * urgency * priority
        total_output_level += output_level


In [ ]:
printed_lines=[]
# Print the assigned tasks and free time per worker
print("Assigned Tasks:")
for worker, tasks in assigned_tasks.items():
    if tasks:
        for task_no, task_type, time_required, priority, urgency in tasks:
            task_count = assigned_tasks[worker].count((task_no, task_type, time_required, priority, urgency))
            line = f"Worker {worker[-1]} - Task No: {task_no}, Task Type: {task_type}, Assigned: {task_count} - Time Required: {task_count * task_time_requirements[task_type]}"
            if line not in printed_lines:
                print(line)
                printed_lines.append(line)

In [ ]:
print("\nFree Time per Worker:")
for worker, time in free_time_per_worker.items():
    print(worker, ":", time)

In [ ]:
print("\nTotal Output Level:", total_output_level)

## 3.1. Unassigned Tasks

In [ ]:
assigned_tasks_all = [item for sublist in assigned_tasks.values() for item in sublist]

# Calculate and print unassigned tasks
unassigned_tasks = []
for _, row in tasks_df.iterrows():
    task_no = row["task_no"]
    task_type = row["task_type"]
    priority = row["priority"]
    urgency = row["urgency"]
    given_count = given_task_counts[(task_type, priority, urgency)]
    assigned_count = sum(1 for (tn, _, _, p, u) in assigned_tasks_all if tn == task_no)
    unassigned_count = given_count - assigned_count
    unassigned_tasks.append((task_no,task_type, priority, urgency, given_count, assigned_count, unassigned_count))

sorted_unassigned_tasks = sorted(unassigned_tasks, key=lambda x: x[0])
print("\nUnassigned Tasks:")
for task_no,task_type, priority, urgency, given, assigned, unassigned in sorted_unassigned_tasks:
    print(f"Task No: {task_no}, Task Type: {task_type}, Priority: {priority}, Urgency: {urgency}, Given: {given}, Assigned: {assigned}, Unassigned: {unassigned}")

# GitHub and Sources

View in Github: ***https://github.com/azminewasi/DynamicTaskAssignmentProblemOM***


## Citation

**Bibtex**:
```
@misc{ wasiDAP2023,
title = {Mathematical Formulation and Implementation of Dynamic Task Assignment Problem in 10 Minute School},
author = {Azmine Touhsik Wasi and Anisha Ahmed and Mahir Absar Khan and Abdur Rahman and Rahatun Nesa Priti},
year = {2023},
url = {https://github.com/azminewasi/DynamicTaskAssignmentProblemOM}
}
```

**APA**:
```
Azmine Touhsik Wasi, Anisha Ahmed, Mahir Absar Khan, Abdur Rahman, & Rahatun Nesa Priti. (2023). Mathematical Formulation and Implementation of Dynamic Task Assignment Problem in 10 Minute School.
```
